In [28]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [29]:
# Load your features and labels datasets
features = pd.read_csv('data/features.csv')
labels = pd.read_csv('data/labels.csv')

# Check the structure of both dataframes to ensure they match
print(features.shape)  # Ensure the number of rows match in features and labels
print(labels.shape)

joined_data = features.join(labels, rsuffix="_target") 
# print(joined_data)

joined_data_long = joined_data.drop(["anchored_target", "heading_target", "rot_target", "sog_target", "cog_target", "latitude_target"], axis=1)
joined_data_lat = joined_data.drop(["anchored_target", "heading_target", "rot_target", "sog_target", "cog_target", "longitude_target"], axis=1)

print(joined_data_long.head())
print(joined_data_lat.head())

joined_data_long.to_csv("data/joined_data_long.csv", index=False)
joined_data_lat.to_csv("data/joined_data_lat.csv", index=False)


(1521377, 8)
(1521377, 7)
   latitude  longitude       cog       sog       rot   heading  anchored  \
0   7.50361   77.58340  0.855833  0.167319 -0.047619  0.877778         0   
1   7.57302   77.49505  0.854444  0.169276  0.039683  0.869444         0   
2   7.65043   77.39404  0.852222  0.165362  0.039683  0.866667         0   
3   7.71275   77.31394  0.855278  0.165362  0.047619  0.869444         0   
4   7.77191   77.23585  0.852778  0.159491  0.055556  0.869444         0   

   time_diff  longitude_target  
0     1393.0          77.49505  
1     1583.0          77.39404  
2     1285.0          77.31394  
3     1259.0          77.23585  
4      901.0          77.18147  
   latitude  longitude       cog       sog       rot   heading  anchored  \
0   7.50361   77.58340  0.855833  0.167319 -0.047619  0.877778         0   
1   7.57302   77.49505  0.854444  0.169276  0.039683  0.869444         0   
2   7.65043   77.39404  0.852222  0.165362  0.039683  0.866667         0   
3   7.71275   7

In [30]:
# Define features (X) and target labels (y)
X_train_long = joined_data_long.drop(columns=["longitude_target"])
y_train_long = joined_data_long["longitude_target"]

X_train_lat = joined_data_lat.drop(columns=["latitude_target"])
y_train_lat = joined_data_lat["latitude_target"]


In [32]:
# Train XGBoost model for longitude prediction
model_long = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, objective='reg:squarederror')
model_long.fit(X_train_long, y_train_long)

# Optionally, evaluate the model
preds = model_long.predict(X_train_long)  # Use test data when available
print(f"Longitude Model Predictions: {preds_long}")


Longitude Model Predictions: [80.22893  80.22893  80.22893  ... 21.002035 21.053448 21.165998]


In [33]:

# Train XGBoost model for latitude prediction
model_lat = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, objective='reg:squarederror')
model_lat.fit(X_train_lat, y_train_lat)

# Optionally, evaluate the model
preds_lat = model_lat.predict(X_train_lat)  # Use test data when available
print(f"Latitude Model Predictions: {preds_lat}")


Latitude Model Predictions: [ 7.7351913  7.7351913  7.7455487 ... 59.699562  59.699562  59.81051  ]


In [36]:
# Load your test dataset
test_data = pd.read_csv('task/ais_test.csv')

# Drop the 'vesselId' column since it's not needed for the predictions
test_data_cleaned = test_data.drop(columns=['vesselId'])

# Convert 'time' column to a timestamp
test_data_cleaned['time'] = pd.to_datetime(test_data_cleaned['time'])

# Convert the 'time' column to a numerical timestep (e.g., seconds since the start of the dataset)
# Assuming you want to count the number of seconds since the first timestamp
test_data_cleaned['time_step'] = (test_data_cleaned['time'] - test_data_cleaned['time'].min()).dt.total_seconds()

# Drop the original 'time' column (if you only want to use 'time_step')
test_data_cleaned = test_data_cleaned.drop(columns=['time'])

# Now the data is ready for predictions
X_test_long = test_data_cleaned.drop(columns=['scaling_factor'])  # Drop scaling_factor if not needed for longitude
preds_test_long = model_long.predict(X_test_long)
print(f"Longitude Predictions on Test Data: {preds_test_long}")

# Similarly for latitude predictions
X_test_lat = test_data_cleaned.drop(columns=['scaling_factor'])  # Drop scaling_factor if not needed for latitude
preds_test_lat = model_lat.predict(X_test_lat)
print(f"Latitude Predictions on Test Data: {preds_test_lat}")

ValueError: feature_names mismatch: ['latitude', 'longitude', 'cog', 'sog', 'rot', 'heading', 'anchored', 'time_diff'] ['ID', 'time_step']
expected anchored, latitude, sog, time_diff, rot, cog, heading, longitude in input data
training data did not have the following fields: time_step, ID